In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
main_directory = "./data"

subdirectories = [f.path for f in os.scandir(main_directory) if f.is_dir()]
dfs = []

for directory in subdirectories:
    files = [f for f in os.scandir(directory) if f.name.endswith('.csv')]
    
    for file in files:
        file_path = file.path
        df_temp = pd.read_csv(file_path)
        x = df_temp['accelerometer_X']
        y = df_temp['accelerometer_Y']
        z = df_temp['accelerometer_Z']
        df_temp['var_x'] = x.var()
        df_temp['std_y'] = y.std(ddof=0)
        df_temp['median_y'] = y.median()
        df_temp['max_x'] = x.max()
        df_temp['min_x'] = x.min()
        df_temp['rms_y'] = np.sqrt((y ** 2).mean())
        df_temp['skewness_y'] = y.skew()
        df_temp['skewness_z'] = z.skew()
        df_temp['kurtosis_z'] = z.kurtosis()
        df_temp['iqr_x'] = x.quantile(0.75) - x.quantile(0.25)
        df_temp['iqr_z'] = z.quantile(0.75) - z.quantile(0.25)
        df_temp['mad_x'] = np.mean(np.abs(x - np.mean(x)))
        df_temp['mad_y'] = np.mean(np.abs(y - np.mean(y)))
        df_temp['mad_z'] = np.mean(np.abs(z - np.mean(z)))
        df_temp['min_index_y'] = y.idxmin()
        
        df_temp['folder'] = os.path.basename(directory)
        df_temp['file'] = os.path.splitext(file.name)[0]
        
        dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True)

df.head()